In [11]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Input, Embedding, LSTM, Dense, Dropout
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Conv1D, MaxPooling1D, GlobalMaxPooling1D, concatenate
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error
import re
from keras.optimizers import Adam
from keras.models import Model

In [14]:
df = pd.read_csv('IMDbe2.csv')

# Preprocess the data
max_features = 5000
maxlen = 1500
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df['review'])
X = tokenizer.texts_to_sequences(df['review'])
X = pad_sequences(X, maxlen=maxlen)
y = df['sentiment'].values
print(y)

['positive' 'positive' 'positive' ... 'negative' 'negative' 'negative']


In [15]:
input_layer = Input(shape=(maxlen,))
embedding_layer = Embedding(max_features, 128)(input_layer)
lstm_layer = LSTM(60, return_sequences=True)(embedding_layer)
conv_layer = Conv1D(60, 5, activation='relu')(embedding_layer)
pool_layer = MaxPooling1D(5)(conv_layer)
gcn_layer = Conv1D(60, 5, activation='relu')(pool_layer)
gcn_layer = GlobalMaxPooling1D()(gcn_layer)
dense_layer = Dense(50, activation='relu')(gcn_layer)
output_layer = Dense(2, activation='softmax')(dense_layer)

In [16]:
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['acc'])

C:\Users\chand\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [17]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1500)]            0         
                                                                 
 embedding_2 (Embedding)     (None, 1500, 128)         640000    
                                                                 
 conv1d_2 (Conv1D)           (None, 1496, 60)          38460     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 299, 60)          0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 295, 60)           18060     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 60)               0         
 balMaxPooling1D)                                          

In [4]:
Y = pd.get_dummies(df['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.15, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1700, 100) (1700, 2)
(300, 100) (300, 2)


In [5]:
history = model.fit(X_train, Y_train, epochs = 10, batch_size=128, verbose = 1)

Epoch 1/10
14/14 [==============================] - 7s 201ms/step - loss: 0.6929 - accuracy: 0.5024
Epoch 2/10
14/14 [==============================] - 3s 194ms/step - loss: 0.6878 - accuracy: 0.6647
Epoch 3/10
14/14 [==============================] - 3s 195ms/step - loss: 0.6282 - accuracy: 0.6565
Epoch 4/10
14/14 [==============================] - 3s 193ms/step - loss: 0.4962 - accuracy: 0.7818
Epoch 5/10
14/14 [==============================] - 3s 192ms/step - loss: 0.3869 - accuracy: 0.8553
Epoch 6/10
14/14 [==============================] - 3s 191ms/step - loss: 0.2809 - accuracy: 0.9088
Epoch 7/10
14/14 [==============================] - 3s 188ms/step - loss: 0.2134 - accuracy: 0.9371
Epoch 8/10
14/14 [==============================] - 3s 190ms/step - loss: 0.1502 - accuracy: 0.9541
Epoch 9/10
14/14 [==============================] - 3s 189ms/step - loss: 0.1171 - accuracy: 0.9712
Epoch 10/10
14/14 [==============================] - 3s 186ms/step - loss: 0.0940 - accuracy: 0.9759

In [1]:
import scipy.stats as stats
import numpy as np

LSTMGCN_acc = [0.9759,0.9712,0.9698,0.9671,0.9729]

mean_acc = np.mean(LSTMGCN_acc)
std_error = stats.sem(LSTMGCN_acc)

print('Mean accuracy: {:.3f} +/- {:.3f}'.format(mean_acc,std_error))

Mean accuracy: 0.971 +/- 0.001
